# Big Data Modeling and Management Assigment - Homework 1

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_.

### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions


### Questions

1. How many different countries exist in the database?
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers? [Hint: 5-node path]
    1. Which `Country` has the most reviews for its beers? [Hint: 5-node path]
1. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV? [Hint: database has NaN values]
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try? (Justify your answer well!) [Hint: database has NaN values]
1. Using Graph Algorithms answer **one** of the following questions: [Hint: make sure to clear the graph before using it again]
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them? [Please use limit of 1000 on beer-review-user path]]
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community? [Please use limit of 1000 on beer-review-user path]]
1. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`? [Hint:inspect two subsets: with and without the beer in question]
    2. Which user is the most influential when it comes to reviews made?
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why? (Justify your answer well!) [Hint: database has NaN values]


Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations or execution. 
 
### Groups  

Groups should have 4 to 5 people.
You should register your group on **moodle**.

### Submission      

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 13**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

In [1]:
from neo4j import GraphDatabase
from pprint import pprint

In [2]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [3]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

### All The Functions you'll need to run queries in Neo4J

In [4]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

### Understanding the Database

In [5]:
query = """
        call db.labels();
    """

result = execute_read(driver, query)

In [6]:
relationshipTypesQuery = """
        CALL db.relationshipTypes();
    """
relationshipTypes = execute_read(driver, relationshipTypesQuery)

pprint(relationshipTypes)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


# Submission

GROUP NUMBER:

**Group27**

GROUP MEMBERS:

| STUDENT NUMBER | STUDENT NAME |
|----------------|--------------|
| Mahmoud Ahmed  | 20221397     |

#### 1. How many different countries exist in the database?

In [7]:
uniqueCountriesQuery = """
    MATCH (c:COUNTRIES)
    RETURN COUNT(DISTINCT c.name) as CountryCount
"""

countriesCount = execute_read(driver, uniqueCountriesQuery)
print(f"There are {countriesCount[0]['CountryCount']} different countries in the database.")

There are 200 different countries in the database.


#### 2. Most reviews:  
    A) Which `Beer` has the most reviews?
    B) Which `Brewery` has the most reviews for its beers?
    C) Which `Country` has the most reviews for its beers?

**A) Which `Beer` has the most reviews?**

__Explanation__

As simple as matching all our BEERS nodes with `REVIEWED` relationship, then grouping the results by `beer.name` and counts the number of REVIEWED relationships for each beer.
then ordering the results in desc order reviews count and returning the most reviewed beer. done ;)

__`WITH clause`__ used here  to aggregate the reviews for each beer before sorting and returning the top result.
__` WITH clause`__ is useful when we want to chain multiple queries together while preserving the results of the first query.

In [8]:
mostReviewedBeerQuery = """
MATCH (b:BEERS)-[r:REVIEWED]-()
WITH b.name as beerName, COUNT(r) AS reviewsCount
ORDER BY reviewsCount DESC
LIMIT 1
RETURN beerName, reviewsCount
"""
mostReviewedBeer = execute_read(driver, mostReviewedBeerQuery)
print(f"The most reviewed beer is {mostReviewedBeer[0]['beerName']} with {mostReviewedBeer[0]['reviewsCount']} reviews.")

The most reviewed beer is IPA with 31387 reviews.


**B) Which `Brewery` has the most reviews for its beers?**

__Explanation__

As simple as matching breweries that have `BREWED` beers that have been `REVIEWED` then again using `WITH clause` to group the results by brewery and count reviews for each brewery.

In [9]:
mostReviewedBreweryQuery = """
MATCH (b:BREWERIES)-[:BREWED]->(beer:BEERS)-[:REVIEWED]-()
WITH b.name as breweryName, COUNT(*) AS reviewsCount
ORDER BY reviewsCount DESC
LIMIT 1
RETURN breweryName, reviewsCount
"""

mostReviewedBrewery = execute_read(driver, mostReviewedBreweryQuery)
print(f"The most reviewed Brewery is {mostReviewedBrewery[0]['breweryName']} with {mostReviewedBrewery[0]['reviewsCount']} reviews on its beers.")

The most reviewed Brewery is Sierra Nevada Brewing Co. with 175161 reviews on its beers.


C) Which `Country` has the most reviews for its beers?

__Explanation__

As simple as matching `COUNTRIES` having at least one `CITIES` with at least one `BREWERY`, which `BREWED` at least one `BEER` that has been `REVIEWED`.  then again using `WITH clause` to group the results by brewery and count reviews for each country and select the top 1 with LIMIT.

In [10]:
mostReviewedCountryQuery = """
MATCH (country:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(brewery:BREWERIES)-[:BREWED]->(beer:BEERS)-[:REVIEWED]-(r)
WITH country.name as countryName, count(r) as reviewsCount
ORDER BY reviewsCount DESC
LIMIT 1
RETURN countryName, reviewsCount
"""

mostReviewedCountry = execute_read(driver, mostReviewedCountryQuery)
print(f"The most reviewed country is {mostReviewedCountry[0]['countryName']} with {mostReviewedCountry[0]['reviewsCount']} reviews on its beers.")

The most reviewed country is US with 7675804 reviews on its beers.


#### 3. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

In [11]:
mostSharedReviewedUsersQuery = """
MATCH (u1:USER {name: 'CTJman'})<-[:POSTED]-(review:REVIEWS)<-[:REVIEWED]-(beer:BEERS)-[:REVIEWED]-(review2:REVIEWS)-[:POSTED]->(u2:USER) WHERE u1 <> u2
WITH u2.name AS userName, count(DISTINCT beer) AS sharedReviews
ORDER BY sharedReviews DESC
LIMIT 10
RETURN userName, sharedReviews
"""

mostSharedReviewedUsers = execute_read(driver, mostSharedReviewedUsersQuery)

print("The top users who have shared reviews with user CTHman are:")

for user in mostSharedReviewedUsers:
    print(user['userName'], "-", user['sharedReviews'])

The top users who have shared reviews with user CTHman are:
acurtis - 1428
Texasfan549 - 1257
kjkinsey - 1205
oline73 - 1191
chippo33 - 1161
mendvicdog - 1156
spycow - 1142
djrn2 - 1122
duceswild - 1081
SocalKicks - 1077


#### 4. Which Portuguese brewery has the most beers?


In [12]:
mostPortugueseBeerProducerBreweryQuery = """
MATCH (country:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(brewery:BREWERIES)-[:BREWED]->(beer:BEERS)
WHERE country.name = 'PT'
WITH brewery, count(beer) as brewedBeers
order by brewedBeers DESC
LIMIT 1
RETURN brewery.name as breweryName, brewedBeers
"""
mostPortugueseBeerProducerBrewery = execute_read(driver, mostPortugueseBeerProducerBreweryQuery)
print(f"The most portuguese beer producer brewery is {mostPortugueseBeerProducerBrewery[0]['breweryName']} with {mostPortugueseBeerProducerBrewery[0]['brewedBeers']} beers.")

The most portuguese beer producer brewery is Dois Corvos Cervejeira with 40 beers.


#### 5. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?


In [13]:
mostSharedReviewedUsersQuery = """
MATCH (country:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(brewery:BREWERIES)-[:BREWED]->(beer:BEERS)-[:REVIEWED]->(review:REVIEWS)
WHERE country.name = 'PT'
WITH brewery, beer, count(review) AS numReviews
ORDER BY numReviews DESC
LIMIT 10
RETURN brewery.name AS breweryName, beer.name AS beerName, numReviews
"""

mostSharedReviewedUsers = execute_read(driver, mostSharedReviewedUsersQuery)
print("The most reviewed beer in Portugal is:")
for row in mostSharedReviewedUsers:
    print(f"Brewery: {row['breweryName']}")
    print(f"Beer: {row['beerName']}")
    print(f"Number of reviews: {row['numReviews']}")
    print()

The most reviewed beer in Portugal is:
Brewery: Unicer União Cervejeira S.A.
Beer: Super Bock
Number of reviews: 391

Brewery: Sociedade Central de Cervejas
Beer: Sagres Cerveja
Number of reviews: 279

Brewery: Unicer União Cervejeira S.A.
Beer: Super Bock Stout
Number of reviews: 82

Brewery: Sociedade Central de Cervejas
Beer: Sagres Cerveja Preta (Dark)
Number of reviews: 77

Brewery: Sociedade Central de Cervejas
Beer: Bohemia
Number of reviews: 67

Brewery: Unicer União Cervejeira S.A.
Beer: Super Bock Abadia
Number of reviews: 37

Brewery: Sociedade Central de Cervejas
Beer: Sagres Mini
Number of reviews: 21

Brewery: Cereuro - Cervejeira Europeia (Tagus)
Beer: Tagus
Number of reviews: 20

Brewery: Unicer União Cervejeira S.A.
Beer: Cristal
Number of reviews: 19

Brewery: Os Três Cervejeiros
Beer: Sovina IPA
Number of reviews: 16



#### 6. On average how many different beer styles does each brewery produce?

In [14]:
averageBeerStylePerBreweryQuery = """
MATCH (brewery:BREWERIES)-[:BREWED]->(beer:BEERS)-[:HAS_STYLE]->(style:STYLE)
WITH brewery, COUNT(DISTINCT style) AS numStyles, COUNT(DISTINCT beer) AS numBeers
RETURN AVG(numStyles) AS avgStylesPerBrewery
"""

averageBeerStylePerBrewery = execute_read(driver, averageBeerStylePerBreweryQuery)
print(f"The average beer styles per Brewery is `{round(averageBeerStylePerBrewery[0]['avgStylesPerBrewery'], 2)}`")

The average beer styles per Brewery is `10.6`


#### 7. Which brewery produces the strongest beers according to ABV?

In [15]:
strongestAbvBeerBreweryQuery = """
MATCH (brewery:BREWERIES)-[:BREWED]->(beer:BEERS)
WHERE beer.abv IS NOT NULL  AND toFloat(beer.abv) >= 0.0
WITH brewery, max(toFloat(beer.abv)) as maxABV
RETURN brewery.name as breweryName, maxABV
ORDER BY maxABV DESC
LIMIT 1
"""

strongestAbvBeerBrewery = execute_read(driver, strongestAbvBeerBreweryQuery)
print(f"The strongest beer with highest abv is brewed in `{strongestAbvBeerBrewery[0]['breweryName']}`")

The strongest beer with highest abv is brewed in `Avondale Brewing Co.`


#### 8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

In [16]:
bestAromaAppearanceBeerQuery = """
MATCH (r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
WHERE r.smell IS NOT NULL AND toFloat(r.smell) >= 0.0
  AND r.look IS NOT NULL AND toFloat(r.look) >= 0.0
WITH s.name AS style, AVG(toFloat(r.smell) + toFloat(r.look)) AS aromaAppearanceScore
ORDER BY aromaAppearanceScore DESC
LIMIT 1
RETURN style, aromaAppearanceScore
"""

bestAromaAppearanceBeer = execute_read(driver, bestAromaAppearanceBeerQuery)
print(f"I would recommend you can try `{bestAromaAppearanceBeer[0]['style']}` beer as it has highest score  `{round(bestAromaAppearanceBeer[0]['aromaAppearanceScore'], 2)}` on aroma & appearance scores")

I would recommend you can try `New England IPA` beer as it has highest score  `8.8` on aroma & appearance scores


#### 9. Using Graph Algorithms answer **one** of the following questions:
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them?
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community?

In [45]:
# Step 0 - Clear graph, graph names need to be unique
try:
    top10SimilarCountriesClearQuery = """
            CALL gds.graph.drop('Countries_Style_Similarity') YIELD graphName;
        """
    top10SimilarCountriesClear = execute_read(driver, top10SimilarCountriesClearQuery)
    pprint(top10SimilarCountriesClear)
except Exception as e:
    pprint(e)

[<Record graphName='Countries_Style_Similarity'>]


In [44]:
# Step 1 - Create an appropriate graph
try:
    top10SimilarCountriesQuery = """
         CALL gds.graph.project.cypher(
            'Countries_Style_Similarity',
            'MATCH (country:COUNTRIES) RETURN id(country) AS id UNION ALL MATCH (style:STYLE) return id(style) as id',

            'MATCH (country:COUNTRIES)<-[]-(city:CITIES)<-[]-(brewery:BREWERIES)-[]->(beer:BEERS)-[]->(style:STYLE)
             WITH country, style, count(*) as similarStylesCount
             ORDER BY similarStylesCount DESC
             WITH country, collect(style)[..10] as top10
             WHERE size(top10) > 9 AND country.name <> "nan"
             UNWIND top10 as top10Countries
             RETURN id(country) as source, id(top10Countries) as target'
        )
        """

    top10SimilarCountries = execute_read(driver, query)
    pprint(top10SimilarCountries)
except Exception as e:
    pprint(e)

[<Record nodeQuery='MATCH (c:COUNTRIES) RETURN id(c) AS id UNION ALL MATCH (s:STYLE) return id(s) as id' relationshipQuery='MATCH (c:COUNTRIES)<-[]-(ci:CITIES)<-[]-(br:BREWERIES)-[]->(be:BEERS)-[]->(s:STYLE)\n             WITH c as country, s as style, count(*) as freq \n             ORDER BY freq DESC \n             WITH country, collect(style)[..10] as top_10 \n             WHERE size(top_10) > 9 and  country.name <>"nan"  UNWIND top_10 as country_top_10 \n             RETURN id(country) as source, id(country_top_10) as target' graphName='Countries_Style_Similarity' nodeCount=313 relationshipCount=1100 projectMillis=1874>]


In [48]:
# Step 2 - Run the algorithm

try:
    query = """
        CALL gds.nodeSimilarity.stream('Countries_Style_Similarity')
            YIELD node1, node2, similarity
            with gds.util.asNode(node1).name AS Country1, gds.util.asNode(node2).name AS Country2, similarity
            RETURN Country1, Country2, similarity
            ORDER BY similarity DESCENDING
            limit 10
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record Country1='KY' Country2='CU' similarity=1.0>,
 <Record Country1='SG' Country2='CU' similarity=1.0>,
 <Record Country1='NZ' Country2='AU' similarity=1.0>,
 <Record Country1='KY' Country2='ET' similarity=1.0>,
 <Record Country1='KY' Country2='SG' similarity=1.0>,
 <Record Country1='SG' Country2='ET' similarity=1.0>,
 <Record Country1='AU' Country2='NZ' similarity=1.0>,
 <Record Country1='CA' Country2='PH' similarity=1.0>,
 <Record Country1='UA' Country2='LT' similarity=1.0>,
 <Record Country1='SG' Country2='KY' similarity=1.0>]


#### 10. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`?
    2. Which user is the most influential when it comes to reviews made?

#### 11. If you had to pick 3 beers to recommend using only this database, which would you pick and why?